<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data')

In [0]:
df.columns = ['party', 'handicapped-infants', 'water-project', 
'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 
'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 
'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 
'south-africa']

In [0]:
df = df.replace({'y':1, 'n':0, '?':np.NaN})

In [19]:
df['handicapped-infants'].value_counts(dropna=False)

0.0    235
1.0    187
NaN     12
Name: handicapped-infants, dtype: int64

In [20]:
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
1,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
4,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [0]:
repub_df = df[df['party']=='republican']

In [0]:
dem_df = df[df['party']=='democrat']

In [0]:
from scipy.stats import ttest_ind

In [32]:
# Null Hypothesis: Dems and Repubs have equal support for budget bill
# Alternative Hypothesis: Dems have greater support for budget bill

ttest_ind(dem_df['budget'], 
          repub_df['budget'], 
          nan_policy='omit')

# T-value favors alternative hypothesis, P-value < .01

Ttest_indResult(statistic=23.12119107755175, pvalue=6.013425749068062e-77)

In [33]:
# Null Hypothesis: Dems and Repubs have equal support for budget bill
# Alternative Hypothesis: Repubs have greater support for budget bill

ttest_ind(repub_df['religious-groups'],
          dem_df['religious-groups'], 
          nan_policy='omit')

# T-value favors alternative hypothesis, P-value < .01

Ttest_indResult(statistic=9.694471882817886, pvalue=3.415425248481181e-20)

In [34]:
# Null Hypothesis: Equivalent support for water project bill
# Alternative Hypothesis: Partisan-biased support for water project bill

ttest_ind(dem_df['water-project'], 
          repub_df['water-project'], 
          nan_policy='omit')

# T-value favors null hypothesis, P-value > .1

Ttest_indResult(statistic=-0.02491808700047811, pvalue=0.9801332440121653)

In [35]:
def t_test(sample_one, sample_two):
  return ttest_ind(sample_one, sample_two, nan_policy='omit')

x = dem_df['mx-missile']
y = repub_df['mx-missile']

# Null Hypothesis: Repubs and Dems have equal support for MX Missile bill
# Alternative Hypothesis: Dems have greater support for MX Missile bill

t_test(x, y)

# T-value favors alternative hypothesis, P-value < .01

Ttest_indResult(statistic=16.371375965888596, pvalue=1.0255175625864558e-46)

In [28]:
mean_table = df.pivot_table(index='party')

mean_table.head()

,aid-to-contras,anti-satellite-ban,budget,crime,duty-free,education,el-salvador-aid,handicapped-infants,immigration,mx-missile,physician-fee-freeze,religious-groups,right-to-sue,south-africa,synfuels,water-project
party,,,,,,,,,,,,,,,,
democrat,0.828897,0.772201,0.888462,0.350195,0.637450,0.144578,0.215686,0.604651,0.471483,0.758065,0.054054,0.476744,0.289683,0.935135,0.505882,0.502092
republican,0.153846,0.242236,0.134969,0.981250,0.090323,0.870130,0.951220,0.189024,0.554878,0.115854,0.987805,0.896970,0.859873,0.655172,0.132075,0.503401
